# Algorytm sympleks

### Programowanie liniowe

Jest to metoda służąca osiągnięciu jak najlepszego rozwiązania w procesie podejmowania decyzji. Programowanie liniowe jest najczęściej stosowanym modelem optymalizacji ze względu na sprawny algorytm obliczeń, a także dzięki możliwości efektownego przedstawienia zagadnień w postaci graficznej. Wadą natomiast jest to, iż nie wszystko można wyrazić za pomocą liczb.

W skład modelu będącego zadaniem programowania liniowego wchodzą:
1. <b>funkcja celu</b> - obrazuje cel, który ma zostać osiągnięty,
2. <b>zmianne decyzyjne</b> - określa narzędzia i zasoby wykorzystane w celu osiągnięcia celu,
3. <b>warunki ograniczające</b> - opisują przeszkody, które mogą pojawić się w trakcie realizacji celu.

Jeżeli model posiada dwie zmienne, łatwo jest go rozwiązać metodą geometryczną w układzie współrzędnych kartezjańskich. Natomiast w przypadku większej liczby zmiennych decyzyjnych, sposób ten się nie sprawdzi. W takiej sytuacji skorzystać można z innej techniki, jaką jest metoda sympleks.

### Metoda sympleks

Nazwa nawiązuje do <b>sympleksu</b>: k-wymiarowym sympleksem nazywamy k-wymiarowy wielościan, który jest otoczką wypukłą swoich k+1 wierzchołków.

![alt text](sympleks.png "Title")

Zadanie programowania liniowego z dowolną liczbą zmiennych można rozwiązać, wyznaczając wszystkie wierzchołkowe punkty wielościanu, a następnie porównując wartości funkcji celu w punktach wierzchołkowych.

W metodzie sympleks można wyróżnić dwa kroki:

<b>Krok 1.</b>
Sprowadzenie warunków ograniczających do układu równań liniowych.

Np. poniższe ograniczenie:
$$2x_{1} + 7x_{2} \leq 13$$

przekształcamy do postaci: $$2x_{1} + 7x_{2} + x_{3} = 13.$$

<b>Krok 2.</b> 
Obliczanie zmiennych decyzyjnych, które przedstawiają najlepszą decyzję dostosowaną do ustalonego modelu.

W związku z wielością punktów powstaje problem wyznaczenia wartości funkcji celu i znalezienie optymalnego wierzchołka, który spełniłby warunek zadania programowania liniowego. Rozwiązanie tego problemy sprowadza się do tego, że znając dowolny wierzchołkowy punkt i wartość funkcji celu w tym punkcie, odrzuca się wszystkie wierzchołki, w których funkcja ta przyjmuje gorsze wartości

W kolejnym kroku, z obecnego wierzchołka, należy przejść krawędzią do sąsiadującego z nim wierzchołka, w którym funkcja celu osiąga lepsze wartości.
Iteracja kończy się w momencie, gdy kolejny przeglądany wierzchołek jest najlepszy pod względem odpowiednich wartości funkcji celu.

![alt text](simplex_alg.png "Title")

### Zastosowania

W działalność gospodarczej, czy też zarządzaniu produkcją dąży się do maksymalnego wykorzystania dostępnych środków w procesie realizacji założonego celu. Zasada ta sprowadza się do zagadnień optymalizacji oraz zasady największej efektywności. Jeżeli z określonych środków osiągany jest maksymalny nakład lub określony cel jest zrealizowany przy jak najmniejszych nakładach, możemy mówić o planie optymalnym. 

Konkretnym przykładem wykorzystania metody sympleks może być problem transportowy. Polega on na znalezieniu takiej drogi od dostawcy towaru do odbiorcy, aby poniesione koszta były jak najmniejsze.